<a href="https://colab.research.google.com/github/Dhanasree-Rajamani/Data-Mining/blob/main/Data%20Mining%20Assignment%202/Timeseries_2_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pycaret 
!pip install pycaret-ts-alpha

In [ ]:
from pycaret.utils import enable_colab
enable_colab()

In [ ]:
import time
import numpy as np
import pandas as pd

from pycaret.datasets import get_data
from pycaret.time_series import *
from pycaret.time_series import TSForecastingExperiment

In [ ]:
!pip install Jinja2
!pip install markupsafe==2.0.1

In [ ]:
fig_kwargs={'renderer': 'notebook'}
fh = 12 
fold = 3

In [ ]:
#file = '/content/drive/MyDrive/DataMining_Datasets/DailyDelhiClimateTrain.csv'
file = 'https://raw.githubusercontent.com/Dhanasree-Rajamani/Data-Mining/main/Data%20Mining%20Assignment%202/DataMining_Datasets/DailyDelhiClimateTrain.csv'
data = pd.read_csv(file)
data.head()

data_set = data
data_set["index"] = pd.to_datetime(data_set["date"])
data_set.drop(columns=["date"], inplace=True)
target = "meantemp"

In [ ]:
data_set.head()

In [ ]:
global_fig_settings = {"renderer": "notebook", "width": 1000, "height": 600}

In [ ]:
exp_exo = TSForecastingExperiment()
exp_exo.setup(
    data=data, target=target, index="index", fh=48,
    numeric_imputation_target="ffill", numeric_imputation_exogenous="ffill",
    fig_kwargs=global_fig_settings, session_id=42
)

In [ ]:
model_exo = exp_exo.create_model("arima", order=(0,1,0), seasonal_order=(0,1,0,24))

In [ ]:
exp_exo.plot_model(model_exo)

In [ ]:
final_model_exo = exp_exo.finalize_model(model_exo)

In [ ]:
def safe_predict(exp, model):
    """Prediction wrapper for demo purposes."""
    try: 
        exp.predict_model(model)
    except ValueError as exception:
        print(exception)
        exo_vars = exp.exogenous_variables
        print(f"{len(exo_vars)} exogenous variables (X) needed in order to make future predictions:\n{exo_vars}")

safe_predict(exp_exo, final_model_exo)

In [ ]:
exp_slim = TSForecastingExperiment()
keep = [target, "index", 'humidity',	'wind_speed',	'meanpressure']
data_slim = data[keep]
exp_slim.setup(
    data=data_slim, target=target, index="index", fh=48,
    numeric_imputation_target="ffill", numeric_imputation_exogenous="ffill",
    fig_kwargs=global_fig_settings, session_id=42 
)

In [ ]:
model_slim = exp_slim.create_model("arima", order=(0,1,0), seasonal_order=(0,1,0,24))

In [ ]:
exp_slim.plot_model(model_slim)

In [ ]:
final_slim_model = exp_slim.finalize_model(model_slim)

In [ ]:
exp_slim.save_model(final_slim_model, "final_slim_model")

In [ ]:
safe_predict(exp_slim, final_slim_model)

In [ ]:
data = pd.read_csv(file)
data.head()

data["index"] = pd.to_datetime(data["date"])
data.drop(columns=["date"], inplace=True)
data.set_index("index", inplace=True)

exog_vars = ['humidity',	'wind_speed',	'meanpressure']
data = data[exog_vars]
data.head()

In [ ]:
exog_exps = []
exog_models = []
for exog_var in exog_vars:
    exog_exp = TSForecastingExperiment()
    exog_exp.setup(
        data=data[exog_var], fh=48,
        numeric_imputation_target="ffill", numeric_imputation_exogenous="ffill",
        fig_kwargs=global_fig_settings, session_id=42
    )

    # Users can customize how to model future exogenous variables i.e. add
    # more steps and models to potentially get better models at the expense
    # of higher modeling time.
    best = exog_exp.compare_models(
        sort="mase", include=["arima", "ets", "exp_smooth", "theta", "lightgbm_cds_dt",]        
    )
    final_exog_model = exog_exp.finalize_model(best)

    exog_exps.append(exog_exp)
    exog_models.append(final_exog_model)

# Step 2: Get future predictions for exog variables ----
future_exog = [
    exog_exp.predict_model(exog_model)
    for exog_exp, exog_model in zip(exog_exps, exog_models)
]
future_exog = pd.concat(future_exog, axis=1)
future_exog.columns = exog_vars

In [ ]:
future_exog

In [ ]:
exp_future = TSForecastingExperiment()

In [ ]:
final_slim_model = exp_future.load_model("final_slim_model")

In [ ]:
!pip install matplotlib==3.1.3

In [ ]:
future_preds = exp_future.predict_model(final_slim_model, X=future_exog)
future_preds.plot()

In [ ]:
data = pd.read_csv(file)
data.head()
data["index"] = pd.to_datetime(data["date"])
data.drop(columns=["date"], inplace=True)
data.set_index("index", inplace=True)

target = "meantemp"
exog_vars = ['humidity',	'wind_speed',	'meanpressure']
include = [target] + exog_vars
data = data[include]
data.head()

In [ ]:
FH=48
metric = "mase"
exclude = ["auto_arima", "bats", "tbats", "lar_cds_dt", "par_cds_dt"]

In [ ]:
exp_auto = TSForecastingExperiment()

# enforce_exogenous=False --> Use multivariate forecasting when model supports it, else use univariate forecasting
exp_auto.setup(
    data=data, target=target, fh=FH, enforce_exogenous=False,
    numeric_imputation_target="ffill", numeric_imputation_exogenous="ffill",
    fig_kwargs=global_fig_settings, session_id=42
)

In [ ]:
#best = exp_auto.compare_models(sort=metric, turbo=False, exclude=exclude)

In [ ]:
model = exp_auto.create_model('lightgbm_cds_dt')

In [ ]:
exp_auto.plot_model(model)

In [ ]:
final_auto_model = exp_auto.finalize_model(model)

In [ ]:
def safe_predict(exp, model):
    """Prediction wrapper for demo purposes."""
    try: 
        future_preds = exp.predict_model(model)
    except ValueError as exception:
        print(exception)
        exo_vars = exp.exogenous_variables
        print(f"{len(exo_vars)} exogenous variables (X) needed in order to make future predictions:\n{exo_vars}")
        
        
        exog_exps = []
        exog_models = []
        for exog_var in exog_vars:
            exog_exp = TSForecastingExperiment()
            exog_exp.setup(
                data=data[exog_var], fh=FH,
                numeric_imputation_target="ffill", numeric_imputation_exogenous="ffill",
                fig_kwargs=global_fig_settings, session_id=42
            )

            # Users can customize how to model future exogenous variables i.e. add
            # more steps and models to potentially get better models at the expense
            # of higher modeling time.
            best = exog_exp.compare_models(
                sort=metric, include=["arima", "ets", "exp_smooth", "theta", "lightgbm_cds_dt",]        
            )
            final_exog_model = exog_exp.finalize_model(best)

            exog_exps.append(exog_exp)
            exog_models.append(final_exog_model)

        # Step 2: Get future predictions for exog variables ----
        future_exog = [
            exog_exp.predict_model(exog_model)
            for exog_exp, exog_model in zip(exog_exps, exog_models)
        ]
        future_exog = pd.concat(future_exog, axis=1)
        future_exog.columns = exog_vars
        
        future_preds = exp.predict_model(model, X=future_exog)
    
    return future_preds  

In [ ]:
future_preds = safe_predict(exp_auto, final_auto_model)
future_preds.plot()